# A simple recommendation system that recommends similar posts for a given post in a social media platform

In [ ]:
#importing the modules and dataset
import pandas as pd
import numpy as np
data_ori= pd.read_csv(r'C:\Users\Asus\Desktop\Imp\ml models\recommendation system\posts.csv')
print(data_ori.head(5))

In [99]:
#checking for null values and removing them
data1=data_ori
print(data1.isnull().sum())
print(data1.shape)

_id            0
title          0
category      28
 post_type     0
dtype: int64
(493, 4)


In [100]:
data2=data1.dropna()
print(data2.isnull().sum())
print(data2.shape)

_id           0
title         0
category      0
 post_type    0
dtype: int64
(465, 4)


In [101]:
#checking the length of the "category" column and merging those values which are less than 2
print(len(data2.category.unique()))

231


In [102]:
category_stats=data2.groupby('category')['category'].agg('count').sort_values(ascending=False)
print(category_stats)

category
Photography                                                                                                      81
Drawings                                                                                                         47
Painting                                                                                                         20
Visual Arts                                                                                                      10
Computer Technology|Machine Learning                                                                              7
Computer Technology|Computer Application|Information Technology                                                   7
Competition Laws                                                                                                  5
Mass Media|Indian Government                                                                                      5
Typography|Pen and ink                                         

In [103]:
print(len(category_stats[category_stats<=1]))

181


In [104]:
Misc=category_stats[category_stats<=1]
data2.category=data2.category.apply(lambda x:'Misc.' if x in Misc else x)
print(data2.head())
print(data2.shape)
print(len(data2.category.unique()))

                        _id                          title          category  \
0  5d62abaa65218653a132c956                    hello there             Misc.   
1  5d6d39567fa40e1417a4931c                      Ml and AI             Misc.   
2  5d7d23315720533e15c3b1ee  What is an Operating System ?             Misc.   
3  5d7d405e5720533e15c3b1f3                     Lord Shiva          Drawings   
4  5d80dfbc6c53455f896e600e   How Competition law evolved?  Competition Laws   

   post_type  
0       blog  
1       blog  
2       blog  
3    artwork  
4       blog  
(465, 4)
51


D:\USER FILE\Softwares_installed\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [105]:
#converting the word vector using TF-IDF Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf= TfidfVectorizer(stop_words='english')
tfidf_matrix=tfidf.fit_transform(data2['category'])
print(tfidf_matrix.shape)

(465, 75)


75 words were used to describe the "category" column for 465 posts

In [106]:
#using Linear Kernel for computing the similarity score
from sklearn.metrics.pairwise import linear_kernel
cosine_sim=linear_kernel(tfidf_matrix,tfidf_matrix)

Other methods for calculating similarity scores are the Euclidean, the Pearson and the Cosine Similarity scores.

In [107]:
#reverse mapping index and title
indices=pd.Series(data2.index,index=data2['title']).drop_duplicates()

In [108]:
#recommendation function
def get_recommendations(title,cosine_sim=cosine_sim):
    idx=indices[title]
    sim_scores=list(enumerate(cosine_sim[idx]))
    sim_scores=sorted(sim_scores, key=lambda x:x[1], reverse=True)
    sim_scores=sim_scores[1:11]
    movie_indices=[i[0] for i in sim_scores]
    return data2['title'].iloc[movie_indices]

In [109]:
get_recommendations('Lord Shiva')

11           Trident
12        Lord shiva
15            Kid Bu
26     Shadow Sketch
30        The Chosen
34           Spartan
45      ROMAN REIGNS
46                OM
47         MK Swords
100         Mahakaal
Name: title, dtype: object

In [110]:
get_recommendations('MIS')

110                            Ethernet
128     What is Information technology?
488          CI/CD using GitHub Actions
106               Topology & it's types
107                Baseband & Broadband
108                  Information System
109    Database management system(DBMS)
112                       Network Model
113    Decision Making with 7 steps !!!
114                Network & it's types
Name: title, dtype: object

In [111]:
get_recommendations('Forms of Cartel.')

5        Raghavan Committee - 2002.
6     Let's discuss some Case laws!
7                  Forms of Cartel.
18     CLASSIFYING FORMS OF CARTELS
0                       hello there
1                         Ml and AI
2     What is an Operating System ?
3                        Lord Shiva
8                      Biodiversity
9                   Economic policy
Name: title, dtype: object

In [ ]:
# The recommendation system gets better when more metadata is taken into consideration.